<td>
   <a target="_blank" href="https://beewant.com" ><img src="https://media.licdn.com/dms/image/C4E0BAQFKjwZohF7zRA/company-logo_200_200/0/1655303818792?e=2147483647&v=beta&t=fmLrigj1HhqskB4jNk6aDb6XASTUx1SnRvMXWBPtuCY"></a>
</td>
<p><B>Welcome to Beewant Platform Tutorial</B></p>

<td>
    <a href="https://www.linkedin.com/in/beewant/" target="_blank">
        <img src="https://img.shields.io/badge/LinkedIn-0077B5?logo=linkedin&logoColor=white" alt="LinkedIn">
    </a>
</td>

<td>
    <a href="https://beewant.slack.com/" target="_blank">
        <img src="https://img.shields.io/badge/Slack-4A154B?logo=slack&logoColor=white" alt="Slack">
    </a>
</td>

<p>Learn how to send annotations for Visual Question Answering (VQA) tasks using the Beewant platform's generative templates. This tutorial provides a comprehensive guide on annotating VQA tasks, enabling you to input detailed responses to visual prompts. Explore the process of sending annotations for VQA tasks, ensuring accurate and relevant answers to visual questions. Follow step-by-step instructions to enhance the annotation process for VQA tasks on the Beewant platform.</p>

In [1]:
import json
import requests
import os
from concurrent.futures import ThreadPoolExecutor

# API Key and Client
Provide a valid api key below in order to properly connect to the Beewant Client.

In [ ]:
# To get your API key go to: Workspace settings -> Account -> Access Token
beewant_api_key = ""

In [ ]:
# The URL to make the API request to
url = "https://beewant.com/api/projects"

# Replace {API_KEY} with your actual API key for authentication
headers = {
    'Authorization': f'Token {beewant_api_key}'
}

# Sending a GET request to the API endpoint with the headers for authentication
response = requests.request("GET", url, headers=headers)

# Check the response status code to verify the request was successful
if response.status_code == 200:
    # If the status code is 200, the request was successful, and we can access the JSON data
    data = response.json()
    print("API response:")
    print(data)
else:
    # If the status code is not 200, there was an error in the request
    print(f"Error: {response.status_code} - {response.text}")

## Data Transformation



In order to prepare the data from the JSON file to fit the format required by the platform, several transformations were applied:

### Constructing Annotation Object:

1. **Image Identification**: The `image` key in each JSON object serves as the identifier for the image associated with the task.

2. **Conversations Parsing**: Each JSON object contains a list of conversations. Assuming the first two elements of this list represent a question and its corresponding answer, they are extracted and used for annotation.

3. **Annotation Object Creation**: For each task, a structured annotation object is constructed. This object consists of two parts: annotations and task data.

    - **Annotations**: Each annotation includes information about the type (e.g., textarea), the text value (e.g., question or answer), and its origin (e.g., manual). Annotations are organized into a list, with each question-answer pair represented as a separate annotation.

    - **Task Data**: This part contains the data associated with the task. In this case, it includes the image data encoded in base64 format.

4. **Image Handling**:
    - **Image Identification**: The image name is extracted from the `image` key of each JSON object.
    - **File Path Construction**: A file path is constructed using the extracted image name and the directory where the images are stored.
    - **Image Existence Check**: The script checks if the image file exists in the specified directory. If not, the task associated with the image is skipped.
    - **Image Encoding**: If the image file exists, it is read and encoded in base64 format. The image format is inferred from the file extension.
    
### Processing File:

The JSON file is processed by iterating through its contents and extracting relevant information for each task. Tasks are constructed based on this information, and only valid tasks (i.e., those with existing images) are included in the final list of task annotation objects.

In [4]:
def construct_annotation_object(image_id, conversations):
    """
    Constructs the annotation object for a given image and its conversations.

    Args:
        image_id (str): The ID or name of the image.
        conversations (list): List of conversations associated with the image.

    Returns:
        dict: Annotation object containing questions and answers.
    """
    annotations = []

    # Iterate through conversations (assuming question-answer pairs)
    for i in range(0, len(conversations), 2):
        if i + 1 < len(conversations):
            # Calculate question ID
            question_id = (i // 2) + 1
            # Extract question and answer from conversations
            question = conversations[i]["value"]
            answer = conversations[i + 1]["value"]

            # Construct question annotation
            question_annotation = {
                "type": "textarea",
                "value": {"text": [question]},
                "origin": "manual",
                "to_name": "image",
                "from_name": f"q{question_id}",
            }

            # Construct answer annotation
            answer_annotation = {
                "type": "textarea",
                "value": {"text": [answer]},
                "origin": "manual",
                "to_name": "image",
                "from_name": f"answer{question_id}",
            }

            # Extend annotations list with question and answer annotations
            annotations.extend([question_annotation, answer_annotation])

    # Construct task object with image ID and annotations
    task_object = {
        "image_id": image_id,
        "annotations": [{
            "result": annotations,
        }],
    }
    return task_object

def process_file_and_construct_task_annotation_objects(file_path):
    """
    Processes a JSON file containing image data and constructs task annotation objects.

    Args:
        file_path (str): Path to the JSON file.

    Returns:
        list: List of task annotation objects.
    """
    # Open and read JSON file
    with open(file_path, 'r') as file:
        data = json.load(file)

    task_annotation_objects = []

    # Iterate through data items and construct task annotation objects
    for item in data:
        image_id = item["id"]
        conversations = item["conversations"]
        task_annotation_objects.append(construct_annotation_object(image_id, conversations))

    return task_annotation_objects

In [ ]:
# Example usage
file_path = 'example_data.json'
constructed_task_annotation_objects = process_file_and_construct_task_annotation_objects(file_path)

## Retrieving Task IDs and Filenames from a Project Endpoint

This function `get_task_ids_and_filenames` is used to fetch task IDs and corresponding filenames from a specified project endpoint.

### Function Overview:

- **Parameters:**
  - `endpoint`: The URL endpoint of the project to retrieve tasks from.
  - `key`: The API key for authorization purposes.

- **Returns:**
  - A dictionary containing task IDs as keys and their respective filenames as values.

### Steps:

1. **Send GET Request:**
   - This function sends a GET request to the specified `endpoint` to retrieve project tasks.

2. **Extract Task Information:**
   - If the response status code is `200` (indicating success), the function extracts task IDs and image URLs from the response JSON data.

3. **Extract Filename:**
   - For each task, it extracts the filename from the image URL using the `os.path.basename` function.

4. **Build Dictionary:**
   - It constructs a dictionary with task IDs as keys and their corresponding filenames as values.

5. **Error Handling:**
   - If the request fails (status code other than `200`), an error message is printed indicating the failure to retrieve project tasks.

6. **Return Dictionary:**
   - The function returns the dictionary containing task IDs and filenames.


In [5]:
# insert the project identifier
project_id = ''

def get_task_ids_and_filenames(endpoint, key):
    task_ids_filenames = {}

    # Send GET request to retrieve project tasks
    response = requests.get(endpoint, headers={'Authorization': f'Token {key}'})

    if response.status_code == 200:
        tasks = response.json()
        for task in tasks:
            task_id = task.get('id')
            image_url = task.get('data', {}).get('image')
            if task_id and image_url:
                # Extract filename from the image URL (data sync from cloud)
                filename = os.path.basename(image_url).split('.')[0]
                task_ids_filenames[task_id] = filename
                #print(f"Task ID: {task_id}, Filename: {filename}")
    else:
        print(f"Failed to retrieve project tasks. Status code: {response.status_code}")

    return task_ids_filenames

# Example usage
endpoint = f"https://beewant.com/api/projects/{project_id}/export"

task_ids_filenames = get_task_ids_and_filenames(endpoint, beewant_api_key)

## Send annotations

This function sends annotations for a single annotation object.

1. **Inputs**:
   - `annotation_obj`: The annotation object containing annotations for a single image.

2. **Process**:
   - Extracts the `image_id` and `annotations` from the provided `annotation_obj`.
   - Finds the matching task ID for the `image_id` in the `task_ids_filenames` dictionary.
   - Sends the annotations to the annotation endpoint if a matching task ID is found.
   - Checks if there are existing annotations for the task ID, and if not, sends the annotations.
   - Prints a success message if the annotations are sent successfully, otherwise prints a failure message.

3. **Example Usage**:
   - Call the function with the annotation object as input to send annotations for a specific image.

In [ ]:
# Function to send annotations for a single annotation object
def send_annotations(annotation_obj):
    image_id = annotation_obj['image_id']
    annotations = annotation_obj['annotations'][0]

    # Find the matching task ID for the current image ID
    matching_task_id = None
    for task_id, task_image_id in task_ids_filenames.items():
        if task_image_id == image_id:
            matching_task_id = task_id
            break

    if matching_task_id is not None:
        # Send annotations to annotation endpoint
        annotation_url = f'https://beewant.com/api/tasks/{matching_task_id}/annotations'
        annotation_response = requests.post(annotation_url,headers={'Authorization': f'Token {beewant_api_key}'}, data=json.dumps(annotations))
        if annotation_response.status_code == 201:
            print(f"Annotations sent successfully for image {image_id}. Task ID: {matching_task_id}")
        else:
            print(f"Failed to send annotations for image {image_id}. Task ID: {matching_task_id}. Status code: {annotation_response.status_code}")

In [ ]:
# Extract filename from the image URL (data sync from cloud)
max_workers = 5  # Adjust the number of threads as needed
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    # Map the send_annotations function to each annotation object in constructed_task_annotation_objects
    # This will execute the function concurrently with multiple threads
    executor.map(send_annotations, reversed(constructed_task_annotation_objects))